In [ ]:
import numpy as np
from scipy.sparse.linalg import eigsh
from numpy import linalg as LA
import pandas
from scipy.interpolate import Akima1DInterpolator
from numpy.linalg import matrix_power
from scipy.linalg import expm
import matplotlib
import matplotlib.pyplot as plt
import time
from scipy import sparse
import sys
from scipy.optimize import minimize
from scipy import optimize
#import openfermion as of
#import fqe
from math import comb
# initialize variables
N = 4   # number of site
half_filling = True

if half_filling == True:

    N_e = N # total number of electrons
    S_z = 0

else:
    N_e = 1
    S_z = 0 # sz here is just up minus down, without the 0.5
    print( ' NOT HALF FILLING CASE RREMEMBER TO INPUT CORRECT SETTING !!!!!!! ')
W = 2*N # each sites can have 2 electrons
# Q is the dimension
Q = pow(4, N)

# decimal number to binary array function
def D2B(num):
    string = f'{num:1b}'
    result = np.zeros(W-len(string), int)

    for ele in string:
        result = np.append(result, int(ele))

    return result #np array , reading from the right hand side


# binary array to decimal function
def B2D(array):
    res = 0
    for ele in array:
        res = (res << 1) | ele
    return res[0]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# N_e list getting a list with correct numberr of electrcons without sz limitation

# create dict for new index in N_e sector
NE_list=[]
for i in range(Q):
    state = D2B(i)
    up_total=0
    down_total=0
    state_4_U = state.reshape((N, 2))
    for j in range(N):
        up_total = up_total + state_4_U[j][0]
        down_total = down_total + state_4_U[j][1]
    if (np.sum(state) == N_e):
        NE_list.append(i)

NE_dimension  = len(NE_list)

In [ ]:
print(NE_dimension)

70


In [ ]:
def check_Hermitian(Ham):
    dimension = len(Ham)
    number=0
    for i in range(dimension):
        for j in range(dimension):
            if i!=j:

                if Ham[i][j]!=np.conjugate(Ham[j][i]):
                    number = number+1
                    print('Error site at i',i,'and j',j,Ham[i][j],'Error site at j',j,'and i',i,Ham[j][i] )
            else:
                if Ham[i][j]!=(Ham[j][i]):
                    number = number+1
                    print('Error site at i',i,'and j',j,Ham[i][j],'Error site at j',j,'and i',i,Ham[j][i] )
    if number == 0:
        print('Correct, this is a Hermitian Matrix ')

def expand_H(input_H):
    result = np.zeros((Q,Q), dtype='complex128')

    for i in range(len(input_H)):

        for j in range(len(input_H)):

            if input_H[i][j]!=0:
                result[ old_index_list[i]  ][ old_index_list[j]    ] = input_H[i][j]


    return result

def get_commutator(matrix1,matrix2):

    result = np.dot(matrix1,matrix2) - np.dot(matrix2,matrix1)

    if np.array_equal(result,np.zeros((   len(matrix1) , len(matrix1) )) ):
                      return ('commute')

    else:

        return result


# this function is to expand vector from sz=something only basis to the full basis
def expand_vec_full_dimen(input_vector):

    full_vector = np.zeros(Q, dtype='complex128')

    for index, value in enumerate(old_index_list):
        full_vector[ value ] = input_vector[index]

    return full_vector


def get_total_spin(input_vector):

    vector_full = expand_vec_full_dimen(input_vector)
    a =  np.dot(s2_op,vector_full)


    #result = np.linalg.norm(a)*np.linalg.norm(a) + np.linalg.norm(b)*np.linalg.norm(b)

    result = np.dot(vector_full,a)
    return result


In [ ]:
lowering_matrix_row_list=[]
lowering_matrix_col_list=[]
# start filling for the U terms, diagonal terms
for index,value in enumerate(NE_list):

    state = D2B(value)
    # reshape it so that each array for one electron
    state_4_U = state.reshape((N, 2))
    for j in range(N):
        hop_state = np.copy(state_4_U) # a copy of the real state so that it won't change the original one
        # up spin part
        if (state_4_U[j][0]==1) and (state_4_U[j][1]==0):
            # same site all the time hence it's sqaure of the either -1 or 1 but both give 1
            hop_state[j][0] = 0
            hop_state[j][1] = 1
            # new number
            hop_state = hop_state.reshape((2*N, 1))
            new_num = B2D(hop_state)
            #print(new_num)
            # for hopping matrix
            lowering_matrix_row_list.append(index)
            lowering_matrix_col_list.append(  NE_list.index(new_num))

# get sprase lowing matrix
lowering_matrix = sparse.csr_matrix((np.ones(len(lowering_matrix_row_list))
                              , (lowering_matrix_row_list, lowering_matrix_col_list)), shape=(NE_dimension, NE_dimension)
                            , dtype=np.float64)


raising_matrix = lowering_matrix.T


sz_matrix_data_list = np.zeros(NE_dimension, dtype='float64')
for index,value in enumerate(NE_list):
    state = D2B(value)
    up_total=0
    down_total=0
    state_4_U = state.reshape((N, 2))
    for j in range(N):
        up_total = up_total + state_4_U[j][0]
        down_total = down_total + state_4_U[j][1]
    sz_matrix_data_list[index] = (up_total-down_total) *1/2


sz_di = np.arange(0,NE_dimension)
sz_matrix = sparse.csr_matrix( ( sz_matrix_data_list , (sz_di, sz_di) )
                              , shape=(NE_dimension, NE_dimension)
                            , dtype=np.float64 )

In [ ]:
s2_op = (lowering_matrix*raising_matrix + raising_matrix*lowering_matrix)/2 + sz_matrix*sz_matrix

In [ ]:
# Set the left one to be up spin and the right one to down spin

In [ ]:
# create dict for new index in N_e sector
old_index_list=[]
for i in range(Q):
    state = D2B(i)
    up_total=0
    down_total=0
    state_4_U = state.reshape((N, 2))
    for j in range(N):
        up_total = up_total + state_4_U[j][0]
        down_total = down_total + state_4_U[j][1]
    if (np.sum(state) == N_e) and (up_total-down_total==S_z):
        old_index_list.append(i)

s_z_dimension  = len(old_index_list)

In [ ]:
print(s_z_dimension)

36


In [ ]:
# shrink s^2 operator

non_0_row_index = s2_op.nonzero()[0]
non_0_col_index = s2_op.nonzero()[1]

# the index of that NE dimension need to go fron index to state representation
non_0_row_state=[]
for i in non_0_row_index:
    non_0_row_state.append (NE_list[i])

non_0_col_state=[]
for i in non_0_col_index:
    non_0_col_state.append (NE_list[i])

s2_op_shrink_row_list = []
s2_op_shrink_col_list = []
s2_op_shrink_data_list = []


for index,value in enumerate(non_0_row_state):

    if (value in old_index_list) and (non_0_col_state[index] in old_index_list):
        new_i_index = old_index_list.index(value)
        new_j_index = old_index_list.index(non_0_col_state[index])

        # record result
        s2_op_shrink_row_list.append(new_i_index)
        s2_op_shrink_col_list.append(new_j_index)
        s2_op_shrink_data_list.append(  s2_op[non_0_row_index[index], non_0_col_index[index] ] )

# get sprase s2 shrink

s2_op_shrink = sparse.csr_matrix( ( s2_op_shrink_data_list , (s2_op_shrink_row_list, s2_op_shrink_col_list) )
                              , shape=(s_z_dimension, s_z_dimension)
                            , dtype=np.float64 )

In [ ]:
# Define the path in Google Drive
if N==9:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/definite_spin_matrix_N={N}_Ne_{N_e}.npz'
else:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/definite_spin_matrix_N={N}.npz'

# Save the sparse matrix
sparse.save_npz(file_path, s2_op_shrink)

In [ ]:
import numpy as np
from scipy.sparse import issparse
from scipy.sparse.linalg import eigsh

# Assuming `s2_op_shrink` is a sparse matrix
s2_op_shrink_dense = s2_op_shrink.toarray()

# Get all eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(s2_op_shrink_dense)
eigenvalues = np.round(eigenvalues, 2)

# Find unique eigenvalues and corresponding indices
unique_eigenvalues, eigenvalue_indices = np.unique(eigenvalues, return_inverse=True)

# Transpose eigenvectors to make iteration easier (each row is an eigenvector)
eigenvectors = eigenvectors.T

# Set check_sign_consistency to True to check for sign consistency of nonzero elements
check_sign_consistency = True

# Dictionary to hold the results for each eigenvalue group
sparsest_info = {}

# Iterate through each unique eigenvalue
for eigenvalue in unique_eigenvalues:
    # Get indices of eigenvectors corresponding to the current eigenvalue
    indices = np.where(eigenvalues == eigenvalue)[0]

    # Extract the corresponding eigenvectors
    vectors = eigenvectors[indices]

    # Iterate over the eigenvectors to find the one with the fewest nonzero elements
    nonzero_counts = []
    nonzero_indices_list = []

    for vec in vectors:
        # Round to 14 decimal places and filter non-zero indices
        vec_rounded = np.round(vec, 14)
        nonzero_indices = np.where(vec_rounded != 0)[0]
        nonzero_indices_list.append(nonzero_indices)
        nonzero_counts.append(len(nonzero_indices))

    # Find the eigenvector with the fewest nonzero elements
    min_nonzero_count = min(nonzero_counts)
    sparsest_index_within_group = nonzero_counts.index(min_nonzero_count)
    sparsest_index_global = indices[sparsest_index_within_group]
    sparsest_nonzero_indices = nonzero_indices_list[sparsest_index_within_group]
    sparsest_nonzero_elements = vectors[sparsest_index_within_group][sparsest_nonzero_indices]

    # Round nonzero elements to 14 decimal places for clarity
    sparsest_nonzero_elements = np.round(sparsest_nonzero_elements, 14)

    # Check if all nonzero elements share the same sign if requested
    all_same_sign = None
    if check_sign_consistency:
        if np.all(sparsest_nonzero_elements > 0):
            all_same_sign = "All Positive"
        elif np.all(sparsest_nonzero_elements < 0):
            all_same_sign = "All Negative"
        else:
            all_same_sign = "Mixed Signs"

    # Store the information for printing later
    sparsest_info[eigenvalue] = {
        "global_index": sparsest_index_global,
        "min_nonzero_count": min_nonzero_count,
        "nonzero_indices": sparsest_nonzero_indices,
        "nonzero_elements": sparsest_nonzero_elements,
        "sign_consistency": all_same_sign,
    }

# Print the details for each eigenvalue's sparsest eigenvector
for eigenvalue, info in sparsest_info.items():
    # Convert arrays to comma-separated strings
    nonzero_indices_str = ", ".join(map(str, info['nonzero_indices']))
    nonzero_elements_str = ", ".join(f"{x:.14f}" for x in info['nonzero_elements'])

    print(f"Eigenvalue: {eigenvalue}")
    print(f"Sparsest eigenvector index in global: {info['global_index']} with {info['min_nonzero_count']} nonzero elements.")
    print(f"Nonzero indices of the sparsest eigenvector: [{nonzero_indices_str}]")
    print(f"Nonzero elements of the sparsest eigenvector: [{nonzero_elements_str}]")

    if check_sign_consistency:
        print(f"Sign consistency: {info['sign_consistency']}")
    print()

Eigenvalue: -0.0
Sparsest eigenvector index in global: 774 with 1 nonzero elements.
Nonzero indices of the sparsest eigenvector: [825]
Nonzero elements of the sparsest eigenvector: [1.00000000000000]
Sign consistency: All Positive

Eigenvalue: 2.0
Sparsest eigenvector index in global: 2837 with 2 nonzero elements.
Nonzero indices of the sparsest eigenvector: [3676, 3679]
Nonzero elements of the sparsest eigenvector: [-0.70710678118655, -0.70710678118655]
Sign consistency: All Negative

Eigenvalue: 6.0
Sparsest eigenvector index in global: 4180 with 6 nonzero elements.
Nonzero indices of the sparsest eigenvector: [858, 860, 861, 883, 884, 886]
Nonzero elements of the sparsest eigenvector: [-0.40824829046386, -0.40824829046386, -0.40824829046386, -0.40824829046386, -0.40824829046386, -0.40824829046386]
Sign consistency: All Negative

Eigenvalue: 12.0
Sparsest eigenvector index in global: 4836 with 20 nonzero elements.
Nonzero indices of the sparsest eigenvector: [952, 955, 957, 958, 976,

In [ ]:
# List to hold the data to save
data_to_save = []

# Iterate through each unique eigenvalue and prepare information for saving
for eigenvalue, info in sparsest_info.items():
    # Create a record with the eigenvalue followed by nonzero indices
    row = [eigenvalue] + info['nonzero_indices'].tolist()  # Ensure nonzero_indices is converted to a list
    data_to_save.append(row)

# Convert the list of lists to a numpy array
data_array = np.array(data_to_save, dtype=object)

# Define the path in Google Drive
if N==9:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/N={N}_sparsest_eigenvectors_info_Ne_{N_e}.txt'
else:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/N={N}_sparsest_eigenvectors_info.txt'

# Save the numpy array to a text file
np.savetxt(file_path, data_array, fmt='%s', delimiter=', ')

print(f"Information about the sparsest eigenvectors has been saved to '{file_path}'.")

Information about the sparsest eigenvectors has been saved to '/content/drive/My Drive/Hubbard_2024/2d/N=8_sparsest_eigenvectors_info.txt'.


In [ ]:
# # reading part

# S = 2.0  # Example: Replace this with the S total value you want to use for initialization

# # Load the saved file with nonzero indices
# file_path = f'/content/drive/My Drive/Hubbard_2024/2d/N={N}_sparsest_eigenvectors_info.txt'
# loaded_data = np.loadtxt(file_path, dtype=str, delimiter=', ')

# # Convert loaded data back to usable form
# # Each row: first value is eigenvalue, followed by nonzero indices
# sparsest_info_load = {}
# for row in loaded_data:
#     eigenvalue = float(row[0])
#     nonzero_indices = list(map(int, row[1:]))
#     sparsest_info_load[eigenvalue] = {
#         "nonzero_indices": nonzero_indices
#     }

# # Initialize `state_to_evolve` based on the loaded eigenvector information
# random_state = False
# state_to_evolve = torch.zeros(len(true_gs), dtype=torch.complex128)

# # Set up PyTorch random number generator
# torch.manual_seed(0)  # Set a fixed random seed for reproducibility

# if random_state:
#     # Random state initialization if random_state is True
#     state_to_evolve_real = torch.rand(len(true_gs), dtype=torch.float64) * 2 - 1
#     state_to_evolve_imag = torch.rand(len(true_gs), dtype=torch.float64) * 2 - 1
#     state_to_evolve = state_to_evolve_real + 1j * state_to_evolve_imag
# else:
#     # Set `state_to_evolve` based on the eigenvector information from the file
#     if S in sparsest_info_load:
#         nonzero_indices = sparsest_info_load[S]["nonzero_indices"]
#         for idx in nonzero_indices:
#             state_to_evolve[idx] = 1  # Set value to 1 for the nonzero indices
#     else:
#         raise ValueError(f"Eigenvalue {S} not found in the saved data. Please check the file and ensure correct S value.")

# # Normalize the state vector
# state_to_evolve = state_to_evolve / torch.norm(state_to_evolve)

# # Move to GPU
# state_to_evolve = state_to_evolve.to(device)

# print(f"State initialized based on eigenvalue {S} for N = {N}.")


In [ ]:
# np.unique(eigenvalues)

In [ ]:
def s2_op(input_state):
    #calculating S+S- only, c.c. gives S-S+. Also, phi S+S- psi can be simplified to just S- onto psi to know the scaling factor,and Sz is easy to get,assuming 0 or 1
    # lowering part
    new_state = np.zeros(len(NE_list))
    for index,value in enumerate(old_index_list): # loop over every computational basis
        state = D2B(value)
        # reshape it so that each array for one electron
        state_4_U = state.reshape((N, 2))
        for j in range(N):
            hop_state = np.copy(state_4_U) # a copy of the real state so that it won't change the original one
            # up spin part
            if (state_4_U[j][0]==1) and (state_4_U[j][1]==0):
                # same site all the time hence it's sqaure of the either -1 or 1 but both give 1
                hop_state[j][0] = 0
                hop_state[j][1] = 1
                # new number
                hop_state = hop_state.reshape((2*N, 1))
                new_num = B2D(hop_state)
                #print(new_num)
                # Correct usage of index method
                try:
                    new_index = NE_list.index(new_num)
                    new_state[new_index] +=  input_state[index]  # Update new_state using the scaling factor
                except ValueError:
                    # Handle the case if new_num is not found in NE_list
                    print(f"Value {new_num} not found in NE_list.")
    value1 = np.round( pow(np.linalg.norm(new_state),2),7)
    # sandwich phi S+ S- phi is the same as measuring the norm of S- psi square
    # raising part
    new_state = np.zeros(len(NE_list))
    for index,value in enumerate(old_index_list): # loop over every computational basis
        state = D2B(value)
        # reshape it so that each array for one electron
        state_4_U = state.reshape((N, 2))
        for j in range(N):
            hop_state = np.copy(state_4_U) # a copy of the real state so that it won't change the original one
            # up spin part
            if (state_4_U[j][0]==0) and (state_4_U[j][1]==1):
                # same site all the time hence it's sqaure of the either -1 or 1 but both give 1
                hop_state[j][0] = 1
                hop_state[j][1] = 0
                # new number
                hop_state = hop_state.reshape((2*N, 1))
                new_num = B2D(hop_state)
                #print(new_num)
                # Correct usage of index method
                try:
                    new_index = NE_list.index(new_num)
                    new_state[new_index] +=  input_state[index]  # Update new_state using the scaling factor
                except ValueError:
                    # Handle the case if new_num is not found in NE_list
                    print(f"Value {new_num} not found in NE_list.")
    value2 = np.round( pow(np.linalg.norm(new_state),2),7)

    return (value1+value2)/2  # this is S(S+1) value -Sz^2

In [ ]:
# # nuemrical check of two methods, agree.
# for i in range(len(eigenvalues)):
#     print(eigenvalues[i],s2_op(eigenvectors[:,i]))